In [ ]:
import pandas as pd
import torch
from sklearn.metrics import log_loss, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler

from deepctr_torch.inputs import SparseFeat, DenseFeat, get_feature_names
from deepctr_torch.models import *
import pandas as pd
import csv
from tqdm import tqdm_notebook as tqdm

In [ ]:
header = open ("/Users/lmac/Documents/inf_search/online-advertising-challenge-fall-2020/train_adv/train.csv").readline ()

In [ ]:
header

In [ ]:
columns = header.strip ().split (";")
columns

In [ ]:
data = pd.read_csv ("/Users/lmac/Documents/inf_search/online-advertising-challenge-fall-2020/train_adv/train.csv", sep=';', names=columns, skiprows=1)


In [ ]:
data

In [ ]:
data['C1'].value_counts()

In [ ]:
data = pd.read_csv ("data/train_2.csv")



In [ ]:
data

In [ ]:
sparse_features = ['C' + str(i) for i in range(1, 30)]
dense_features = ['I' + str(i) for i in range(1, 2)]

data[sparse_features] = data[sparse_features].fillna('-1', )
data[dense_features] = data[dense_features].fillna(0, )
target = ['Label']



In [ ]:
# 1.Label Encoding for sparse features,and do simple Transformation for dense features
for feat in sparse_features:
        lbe = LabelEncoder()
        data[feat] = lbe.fit_transform(data[feat])
mms = MinMaxScaler(feature_range=(0, 1))
data[dense_features] = mms.fit_transform(data[dense_features])



In [ ]:
# 2.count #unique features for each sparse field,and record dense feature field name

fixlen_feature_columns = [SparseFeat(feat, data[feat].nunique())
                              for feat in sparse_features] + [DenseFeat(feat, 1, )
                                                              for feat in dense_features]

dnn_feature_columns = fixlen_feature_columns
linear_feature_columns = fixlen_feature_columns

feature_names = get_feature_names(linear_feature_columns + dnn_feature_columns)



In [ ]:
# 3.generate input data for model

train, test = train_test_split(data, test_size=0.2, random_state=2020)
train_model_input = {name: train[name] for name in feature_names}
test_model_input = {name: test[name] for name in feature_names}
train_model_all = {name: data[name] for name in feature_names}

In [ ]:
train

In [ ]:
import numpy as np
np.sum(train_model_input[target].values)
len(train_model_input[target].values)

In [ ]:
train_model_input

In [ ]:
# 4.Define Model,train,predict and evaluate

device = 'cpu'
use_cuda = False
if use_cuda and torch.cuda.is_available():
        print('cuda ready...')
        device = 'cuda:0'

model = DeepFM(linear_feature_columns=linear_feature_columns, dnn_feature_columns=dnn_feature_columns,
                   task='binary',
                   l2_reg_embedding=1e-5, device=device)

model.compile("adagrad", "binary_crossentropy",
                  metrics=["binary_crossentropy", "auc"], )

history = model.fit(train_model_all, data[target].values, batch_size=32, epochs=10, verbose=2,
                        validation_split=0)


In [ ]:
import zlib
from collections import defaultdict

HEADER = "timestamp;label;C1;C2;C3;C4;C5;C6;C7;C8;C9;C10;CG1;CG2;CG3;l1;l2;C11;C12".split(';')
COUNT_BASIC = 2
COUNT_CATEG_FIRST = 10
COUNT_GROUP = 3
COUNT_CATEG_SECOND = 2
COUNT_CATEGORIAL = COUNT_CATEG_FIRST+COUNT_CATEG_SECOND

OUT_HEADER = "timestamp;Label;I1;I2;C1;C2;C3;C4;C5;C6;C7;C8;C9;C10;C11;C12;C13;C14;C15;C16;C17;C18;C19;C20;C21;C22;C23;C24;C25;C26;C27;C28;C29;C30".split(';')

CG1_GROUPS = {"C13": [335, 99, 122, 58, 385, 268, 341, 382, 52, 416],
              "C14": [332, 76, 59, 96, 399, 273, 57, 53, 49, 437],
              "C15": [334, 155, 422, 130, 74, 357, 419, 276, 54, 343],
              "C16": [333, 154, 331, 151, 18, 279, 330, 449, 336, 269],
              "C17": [139, 412, 123, 124, 150, 205, 435, 277, 438, 88],
              "C18": []} #For other vals

CG2_GROUPS = {"C19": [16810, 20892, 25731, 8426, 8395, 29347, 18705, 2390, 8833, 823],
              "C20": [30009, 16444, 22582, 29432, 3864, 4378, 25900, 7326, 31328, 3326],
              "C21": [2293, 944, 19883, 29463, 6746, 2253, 16676, 7923, 516, 17179],
              "C22": [18714, 7636, 14322, 10328, 6619, 29768, 15650, 30235, 24843, 18543],
              "C23": [28695, 9793, 2254, 19225, 20755, 3746, 30749, 24338, 207, 18724],
              "C24": []}

CG3_GROUPS = {"C25": [46839, 49517, 49513, 48114, 44289, 57895, 17419, 11599, 56445, 7592],
              "C26": [38344, 46594, 6214, 49272, 45902, 46401, 24887, 3311, 49784, 37588],
              "C27": [45509, 75, 15769, 38346, 27636, 28676, 40461, 5178, 19563, 8894],
              "C28": [56597, 15529, 45501, 18336, 49966, 21144, 47396, 2340, 49518, 8650],
              "C29": [33701, 54846, 55412, 43844, 43845, 20076, 49962, 43666, 23339, 33789],
              "C30": []}


def ProcessGroup(group_name, stat, group_feature):

    vals_array = None
    if group_name == 'CG1':
        vals_array = CG1_GROUPS
    elif group_name == 'CG2':
        vals_array = CG2_GROUPS
    elif group_name == 'CG3':
        vals_array = CG3_GROUPS
    else:
        raise Exception("Unknown group")

    result = defaultdict(list)
    for val in group_feature.split(','):
        if val == '':
            continue
        else:
            val = int(val)

        for key, feature_vals in vals_array.items():
            if len(feature_vals) == 0: # orphan
                result[key].append(str(val))
                break
            if val in feature_vals:
                result[key].append(str(val))
                break
    for key in result:
        if len(result[key]) == 0:
            result[key] = ''
        else:
            result[key] = str(zlib.crc32((",".join(sorted(result[key]))).encode()))
        global_stat[key+"-"+result[key]] += 1
    return result

def GetFeatures(input_str, global_statistics):
    features = {}
    for feature_id, feature_str in enumerate(input_str.split(';')):
        feature_name = HEADER[feature_id]
        if feature_name == 'label':
            feature_name = 'Label'
        elif feature_name.startswith('CG'): # group
            features.update(ProcessGroup(feature_name, global_stat, feature_str))
        elif feature_name.startswith('l'): # counters
            feature_name = feature_name.replace('l', 'I')

        if not feature_name.startswith('CG'):
            features.update({feature_name: feature_str})
        if feature_name.startswith('C') and not feature_name.startswith('CG'):
            global_statistics[feature_name+"-"+feature_str] += 1

    return features

def CreateOutFeatures(input_str, statistics):
    features = GetFeatures(input_str, statistics)
    out = ""
    for feature_name in OUT_HEADER:
        feature_val = features.get(feature_name)
        if feature_val is None:
            feature_val = ''
        out += feature_val + ","
    return out[:-1]

def ProcessFile(filename_in, filename_out, global_statistics):
    with open(filename_in, 'r', encoding='utf-8') as input, \
            open(filename_out, 'w', encoding='utf-8') as output:
        output.write(','.join(OUT_HEADER)+'\n')
        for counter, line in enumerate(input):
            if line.startswith('timestamp'):
                continue

            line = line[:-1]
            out_line = CreateOutFeatures(line, global_statistics)
            output.write(out_line+'\n')
            if counter % 100000 == 0:
                print(filename_in, counter)
                #break

global_stat = defaultdict(int)
ProcessFile('/Users/lmac/Documents/inf_search/online-advertising-challenge-fall-2020/train_adv/train.csv', 'data/train_2.csv', global_stat)
ProcessFile('/Users/lmac/Documents/inf_search/online-advertising-challenge-fall-2020/test_adv/test.csv', 'data/test_2.csv', global_stat)

sorted_stat = sorted(global_stat.items(), key=lambda kv: -kv[1])
with open('data/stat_2.txt', 'w', encoding='utf-8') as stat_file:
    for stat_name, stat_count in sorted_stat:
        stat_file.write(stat_name+"\t"+str(stat_count)+"\n")

In [ ]:
with open('/Users/lmac/Documents/inf_search/online-advertising-challenge-fall-2020/train_adv/train.csv', newline='') as csvfile, open('data/train_drop_groups.csv', 'w', newline='') as csvout: 
    spamreader = csv.reader(csvfile, delimiter=';', quotechar='|')       
    spamwriter = csv.writer(csvout, delimiter=';',
                            quotechar='|', quoting=csv.QUOTE_MINIMAL)
    for row in spamreader:
        out_row = row[0:12]
        out_row.extend(row[15:])
        spamwriter.writerow(out_row)

In [ ]:
with open('data/train_drop_groups.csv', newline='') as csvfile, open('data/train_libffm', 'wb') as f: 
    spamreader = csv.reader(csvfile, delimiter=';', quotechar='|')
    hdr = next(spamreader)
    indices = list(map(str, range(0, len(hdr) - 1)))
    c_i = 0
    for row in spamreader:
        if (row[1] == '1') | (c_i < 1000000):
            out_line = row[1] + " " 
            row = [row[0]] + row[2:]
            zipper = list(zip(indices, row))
            out_line += " ".join([a + ":1:" + b for a,b in zipper[0:1]]) + " "
            out_line += " ".join([a + ":" + b + ":1" for a,b in zipper[1:11]])+ " "
            out_line += " ".join([a + ":1:" + b for a,b in zipper[11:13]])+ " "
            out_line += " ".join([a + ":" + b + ":1" for a,b in zipper[13:]])+ " "
            f.write(str.encode(out_line[:-1] + '\n'))
            c_i +=1

In [ ]:
import xlearn as xl

ffm_model = xl.create_ffm()
ffm_model.setTrain("data/train_libffm")
ffm_model.setValidate("data/test_libffm_small")
ffm_model.setTest('data/test_libffm')
ffm_model.setSigmoid()
ffm_model.disableEarlyStop();
param = {'task':'binary', 'lr':0.2, 'lambda':0.002, 'epoch':10}

ffm_model.fit(param, "data/model.out")

In [ ]:
with open('/Users/lmac/Documents/inf_search/online-advertising-challenge-fall-2020/test_adv/test.csv', newline='') as csvfile, open('data/test_drop_groups.csv', 'w', newline='') as csvout: 
    spamreader = csv.reader(csvfile, delimiter=';', quotechar='|')       
    spamwriter = csv.writer(csvout, delimiter=';',
                            quotechar='|', quoting=csv.QUOTE_MINIMAL)
    for row in spamreader:
        out_row = row[0:12]
        out_row.extend(row[15:])
        spamwriter.writerow(out_row)

In [ ]:
with open('data/test_drop_groups.csv', newline='') as csvfile, open('data/test_libffm_small', 'wb') as f: 
    spamreader = csv.reader(csvfile, delimiter=';', quotechar='|')
    hdr = next(spamreader)
    indices = list(map(str, range(0, len(hdr) - 1)))
    c_i = 0
    for row in spamreader:
        if (c_i < 10000):
            out_line = row[1] + " " 
            row = [row[0]] + row[2:]
            zipper = list(zip(indices, row))
            out_line += " ".join([a + ":1:" + b for a,b in zipper[0:1]]) + " "
            out_line += " ".join([a + ":" + b + ":1" for a,b in zipper[1:11]])+ " "
            out_line += " ".join([a + ":1:" + b for a,b in zipper[11:13]])+ " "
            out_line += " ".join([a + ":" + b + ":1" for a,b in zipper[13:]])+ " "
            f.write(str.encode(out_line[:-1] + '\n'))
            c_i += 1

In [ ]:
header = open ("data/test_2.csv").readline ()

In [ ]:
header

In [ ]:
with open('data/train_2.csv', newline='') as csvfile, open('data/train_libffm', 'wb') as f: 
    spamreader = csv.reader(csvfile, delimiter=',', quotechar='|')
    hdr = next(spamreader)
    indices = list(map(str, range(0, len(hdr) - 2)))
    c_i = 0
    c_sum = 0
    for row in spamreader:
        if ((row[1] == '1') | (c_i < 1000000)):
            out_line = row[1] + " " 
            c_sum += (row[1] == '1')*1
            row = row[2:]
            zipper = list(zip(indices, row))
            out_line += " ".join([a + ":1:" + b for a,b in zipper[0:2]]) + " "
            out_line += " ".join([a + ":" + b + ":1" for a,b in zipper[2:31]])+ " "
            f.write(str.encode(out_line[:-1] + '\n'))
            c_i = c_i + 1


In [ ]:
import xlearn as xl

ffm_model = xl.create_ffm()
ffm_model.setTrain("data/train_libffm")
#ffm_model.setValidate("./small_test.txt")
#ffm_model.disableEarlyStop();
param = {'task':'binary', 'lr':0.2, 'lambda':0.002, 'epoch':10}

ffm_model.fit(param, "data/model.out")

In [ ]:
from datetime import datetime
from csv import DictReader
from math import exp, log, sqrt


# parameters #################################################################

train = 'data/train_2.csv'  # path to training file
test = 'data/test_2.csv'  # path to testing file

D = 2 ** 20   # number of weights use for learning
alpha = .1    # learning rate for sgd optimization

In [ ]:

def logloss(p, y):
    p = max(min(p, 1. - 10e-12), 10e-12)
    return -log(p) if y == 1. else -log(1. - p)



def get_x(csv_row, D):
    x = [0]  # 0 is the index of the bias term
    for key, value in csv_row.items():
        index = int(value + key[1:], 31) % D  
        x.append(index)
    return x  # x contains indices of features that have a value of 1



def get_p(x, w):
    wTx = 0.
    for i in x:  # do wTx
        wTx += w[i] * 1.  # w[i] * x[i], but if i in x we got x[i] = 1.
    return 1. / (1. + exp(-max(min(wTx, 20.), -20.)))  # bounded sigmoid



def update_w(w, n, x, p, y):
    for i in x:
        # alpha / (sqrt(n) + 1) is the adaptive learning rate heuristic
        # (p - y) * x[i] is the current gradient
        # in our case, if i in x then x[i] = 1
        w[i] -= (p - y) * alpha / (sqrt(n[i]) + 1.)
        n[i] += 1.

    return w, n

In [ ]:
# training and testing #######################################################

# initialize our model
w = [0.] * D  # weights
n = [0.] * D  # number of times we've encountered a feature

# start training a logistic regression model using on pass sgd
loss = 0.
for t, row in enumerate(DictReader(open(train))):
    y = 1. if row['Label'] == '1' else 0.

    del row['Label']  
    del row['timestamp']  

    # main training procedure
    # step 1, get the hashed features
    x = get_x(row, D)

    # step 2, get prediction
    p = get_p(x, w)

    # for progress validation, useless for learning our model
    loss += logloss(p, y)
    if t % 1000000 == 0 and t > 1:
        print('%s\tencountered: %d\tcurrent logloss: %f' % (
            datetime.now(), t, loss/t))

    # step 3, update model with answer
    w, n = update_w(w, n, x, p, y)



In [ ]:
# testing (build kaggle's submission file)
c_sum = 0
with open('submission1234.csv', 'w') as submission:
    submission.write('Id,Click\n')
    for t, row in enumerate(DictReader(open(test))):
        Id = t + 1
        del row['Label']
        del row['timestamp']
        x = get_x(row, D)
        p = get_p(x, w)
        if p > 0.5:
            c_sum += 1
        submission.write('%s,%f\n' % (Id, p))
        if t % 1000000 == 0 and t > 1:
            print('%s\tencountered: %d\n' % (datetime.now(), t))

In [ ]:
c_sum

In [ ]:
for t, row in enumerate(DictReader(open(train))):
    print(t)
    print(row)
    if t == 1:
        break
        

In [ ]:
from datetime import datetime
from csv import DictReader

def csv_to_vw(loc_csv, loc_output, train=True):

    c_sum = 0
    start = datetime.now()
    print("\nTurning %s into %s. Is_train_set? %s"%(loc_csv,loc_output,train))

    with open(loc_output,"w") as outfile:
        for e, row in enumerate( DictReader(open(loc_csv)) ):
            
            #Creating the features
            numerical_features = ""
            categorical_features = ""
            for k,v in row.items():
                if k not in ["Label","timestamp"]:
                    if "I" in k: # numerical feature, example: I5
                        if len(str(v)) > 0: #check for empty values
                            numerical_features += " %s:%s" % (k,v)
                    if "C" in k: # categorical feature, example: C2
                        if len(str(v)) > 0:
                            categorical_features += " %s" % v
            
            #Creating the labels
            if train: 
                if row['Label'] == "1":
                    label = 1
                    c_sum += 1
                else:
                    label = -1 #we set negative label to -1
                outfile.write( "%s '%s |i%s |c%s\n" % (label,row['timestamp'],numerical_features,categorical_features) )
            
            else: #we dont care about labels
                outfile.write( "1 '%s |i%s |c%s\n" % (row['timestamp'],numerical_features,categorical_features) )
            
            #Reporting progress
            if e % 1000000 == 0:
                 print("%s\t%s"%(e, str(datetime.now() - start)))
            
    print("\n %s Task execution time:\n\t%s"%(e, str(datetime.now() - start)))
    print(c_sum)

In [ ]:
csv_to_vw("data/train_2.csv", "data/train_2.vw",train=True)
csv_to_vw("data/test_2.csv", "data/test_2.vw",train=False)

In [ ]:
import math

def zygmoid(x):
	return 1 / (1 + math.exp(-x))

with open("kaggle.click.submission.csv","w") as outfile:
    outfile.write("Id,Click\n")
    i = 1
    for line in open("data/click.preds.txt"):
             row = line.strip().split(" ")
             outfile.write("%s,%f\n"%(i,zygmoid(float(row[0]))))
             i = i + 1

In [ ]:
sub_r = pd.read_csv ("submission1234.csv")



In [ ]:
sub_r

In [ ]:
sorted(sub_r['Click'], reverse=True)

In [ ]:
sorted(sub_r['Click'])

In [ ]:
sub_r['Click'].values[sub_r['Click'] < 0.01] = 0

In [ ]:
sub_r.to_csv("submission12345.csv", index = False)